# 处理数据，生成一个大的npy文件
导入os、numpy、PIL库，并确保tqdm库可用。

In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [18]:
import os
import glob
from PIL import Image, ImageEnhance
import numpy as np

# 定义输入图像和掩码目录
image_dir = "data/Img/"
mask_dir = "data/Lab/"

# 创建输出目录
output_image_dir = "data/Img_"
output_mask_dir = "data/Lab_"
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# 获取所有图像和掩码文件
image_files = glob.glob(os.path.join(image_dir, "*.jpg"))
mask_files = glob.glob(os.path.join(mask_dir, "*.png"))

# 检查图像和掩码的数量是否匹配
if len(image_files) != len(mask_files):
    raise ValueError("Number of images and masks do not match!")

# 添加自定义变换，每种变换都应用一次
def augment_image_and_mask(image, mask, base_name):
    augmented_images = []
    augmented_masks = []
    
    # 1. 垂直翻转
    flipped_img = image.transpose(method=Image.FLIP_LEFT_RIGHT)
    flipped_mask = mask.transpose(method=Image.FLIP_LEFT_RIGHT)
    augmented_images.append((flipped_img, flipped_mask, "_flip"))

    # 2. 对比度增强
    contrast_factor = np.random.uniform(1.0, 1.5)
    contrasted_img = ImageEnhance.Contrast(image).enhance(contrast_factor)
    augmented_images.append((contrasted_img, mask, "_contrast"))

    # 3. 亮度增强
    brightness_factor = np.random.uniform(0.5, 1.5)
    brightened_img = ImageEnhance.Brightness(image).enhance(brightness_factor)
    augmented_images.append((brightened_img, mask, "_brightness"))

    # 4. 放大和裁切
    scale_factor = 1.2
    new_size = (int(image.size[0] * scale_factor), int(image.size[1] * scale_factor))
    resized_img = image.resize(new_size)
    resized_mask = mask.resize(new_size)
    augmented_images.append((resized_img, resized_mask, "_resize"))

    # 5. 裁切85%
    cropped_img = image.crop((0, 0, int(image.size[0] * 0.85), int(image.size[1] * 0.85)))
    cropped_mask = mask.crop((0, 0, int(mask.size[0] * 0.85), int(mask.size[1] * 0.85)))
    augmented_images.append((cropped_img, cropped_mask, "_crop"))

    # 6. 添加椒盐噪声
    image_np = np.array(image)
    height, width, channel = image_np.shape
    num_salt = 500  # 椒盐噪声的数量
    for i in range(num_salt):
        x = np.random.randint(0, height)
        y = np.random.randint(0, width)
        image_np[x, y, :] = 255  # 椒盐噪声
    noisy_img = Image.fromarray(image_np)
    augmented_images.append((noisy_img, mask, "_salt_noise"))

    # 7. 最终调整为256x256
    resized_200_img = image.resize((200, 200))
    resized_200_mask = mask.resize((200, 200))
    augmented_images.append((resized_200_img, resized_200_mask, "_final_resize"))

    # 保存所有增强后的图像和掩码
    for img, msk, suffix in augmented_images:
        new_img_name = base_name + suffix + ".jpg"
        new_mask_name = base_name + suffix + ".png"

        img.save(os.path.join(output_image_dir, new_img_name))
        msk.save(os.path.join(output_mask_dir, new_mask_name))

# 逐一处理图像和掩码
for img_file, mask_file in zip(image_files, mask_files):
    img = Image.open(img_file).convert('RGB')
    mask = Image.open(mask_file).convert('L')

    base_name = os.path.splitext(os.path.basename(img_file))[0]
    augment_image_and_mask(img, mask, base_name)

print("Data augmentation completed for images and masks!")

Data augmentation completed for images and masks!


In [20]:
# 文件夹路径
image_dir = 'data/Img'
mask_dir = 'data/Lab'

# 获取文件列表
images = sorted(os.listdir(image_dir))
masks = sorted(os.listdir(mask_dir))

# 确保文件对应
assert len(images) == len(masks), "图片和掩码数量不一致"

# 完整路径
image_paths = [os.path.join(image_dir, img) for img in images]
mask_paths = [os.path.join(mask_dir, msk) for msk in masks]

# 分割数据集
train_imgs, test_imgs, train_masks, test_masks = train_test_split(
    image_paths, mask_paths, test_size=0.2, random_state=42
)

def load_images(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('RGB')
        img_array = np.array(img)
        img_array = np.transpose(img_array, (2, 0, 1))  # 将通道轴移到最前面
        data.append(img_array)
    return np.array(data)

def load_masks(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('L')
        data.append(np.array(img))
    return np.array(data)

# 加载并保存训练集
train_images = load_images(train_imgs)
train_masks = load_masks(train_masks)
np.save('data/train_images.npy', train_images)
np.save('data/train_masks.npy', train_masks)

# 加载并保存测试集
test_images = load_images(test_imgs)
test_masks = load_masks(test_masks)
np.save('data/test_images.npy', test_images)
np.save('data/test_masks.npy', test_masks)

# 运行

In [8]:
!python main.py

Parameters:
{'work_dir': './work_dir/', 'config': './config/train.yaml', 'phase': 'train', 'save_score': False, 'seed': 1, 'feeder': 'feeder.Feeder', 'num_worker': 8, 'train_feeder_args': {'mode': 'train', 'data_path': './data/train_images.npy', 'label_path': './data/train_masks.npy'}, 'test_feeder_args': {'mode': 'test', 'data_path': './data/test_images.npy', 'label_path': './data/test_masks.npy'}, 'model': 'net.self_net', 'model_args': {'num_class': 4, 'channel': 3}, 'weights': None, 'ignore_weights': [], 'base_lr': 0.01, 'step': [10, 50], 'device': 0, 'optimizer': 'Adam', 'nesterov': True, 'batch_size': 16, 'test_batch_size': 16, 'start_epoch': 0, 'num_epoch': 150, 'weight_decay': 0.0005}

Parameters: 147300
Epoch:[1/150]
100%|███████████████████████████████████████████| 40/40 [00:03<00:00, 12.27it/s]
Train iou: [0.00216127 0.26110874 0.0246982 ] Train miou: 0.09598940342272322 Mean loss: 0.6753625631332397 LR: [0.0095]
  0%|                                                    | 0/10

清理log

In [7]:
!rm -rf work_dir/*